# Lesson 4.2: Tools in LangChain

---

In Lesson 4.1, we introduced **Agents** and how they use Large Language Models (LLMs) for decision-making and planning. An indispensable part of an Agent's capabilities is the use of **Tools**. Tools allow Agents to interact with the external world and perform specific tasks beyond the pure text generation capabilities of an LLM.

## 1. Concept of Tools

### 1.1. What are Tools?

**Tools** in LangChain are functions or APIs that an Agent can invoke to perform specific actions. They serve as the interface for the Agent to interact with external systems, gather information, or perform calculations.

* **Relationship:** If the LLM is the "brain" of the Agent, then Tools are its "hands and feet." The LLM thinks about the problem and decides what needs to be done, then it selects an appropriate Tool to perform that action.

* **Important Description:** Each Tool has a clear textual description of its functionality. The LLM will read this description to understand what the Tool does and when to use it.



### 1.2. Why are Tools Needed?

* **Extend LLM Capabilities:** LLMs have vast knowledge but cannot perform tasks like real-time searching, complex calculations, reading files, or making API calls. Tools fill this gap.

* **Up-to-date Information:** LLMs have static knowledge up to their training cutoff. Tools (e.g., web search) allow the Agent to access the latest information.

* **Perform Actions:** Enable the Agent to interact with other systems (e.g., sending emails, updating databases).

* **Reduce Hallucinations:** By using Tools to fetch factual information, the Agent can provide more accurate and less fabricated answers.


---

## 2. Creating Custom Tools

You can create custom tools for your Agent to perform any function you define. LangChain provides two main ways to do this: using the `tool` decorator or inheriting from `BaseTool`.

### 2.1. Using the `tool` decorator (Simplest Way)

The simplest way to create a Tool is to use the `@tool` decorator from `langchain.tools`. It will automatically turn a Python function into a Tool.

* **Characteristics:**
    * The function must have a clear docstring describing its functionality. This docstring will be used by the LLM to decide when to call the Tool.
    * The function takes a single argument (usually a string) as the Tool's input.

In [ ]:
# Install the library if not already installed
# pip install langchain

from langchain.tools import tool

@tool
def get_current_weather(location: str) -> str:
    """
    Get the current weather information for a specific location.
    Input is the city or location name.
    Example: "Hanoi", "London", "New York".
    """
    if "Hanoi" in location:
        return "Hanoi Weather: 28°C, sunny, 70% humidity."
    elif "London" in location:
        return "London Weather: 15°C, cloudy, light rain."
    else:
        return "Weather information not found for this location."

# You can test the Tool like a regular function
print(get_current_weather("Hanoi"))
print(get_current_weather("Da Nang"))

### 2.2. Inheriting from `BaseTool` (More Granular Control)

If you need more granular control over the Tool (e.g., input validation with Pydantic, complex error handling), you can inherit from the `BaseTool` class.

In [ ]:
# Install the library if not already installed
# pip install langchain pydantic

from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class WeatherToolInput(BaseModel):
    """Input for WeatherTool."""
    location: str = Field(description="City or location name to get weather information.")

class CustomWeatherTool(BaseTool):
    name = "get_weather_information"
    description = "Useful when you need to get the current weather information for a specific location."
    args_schema: Type[BaseModel] = WeatherToolInput # Define input schema

    def _run(self, location: str) -> str:
        """Use the tool synchronously."""
        if "Hanoi" in location:
            return "Hanoi Weather: 28°C, sunny, 70% humidity."
        elif "London" in location:
            return "London Weather: 15°C, cloudy, light rain."
        else:
            return "Weather information not found for this location."

    async def _arun(self, location: str) -> str:
        """Use the tool asynchronously."""
        # In this simple example, we just call the synchronous _run
        return self._run(location)

# Initialize the Tool
custom_weather_tool = CustomWeatherTool()

# You can test the Tool
print(custom_weather_tool.run("Hanoi"))


---

## 3. Using LangChain's Built-in Tools

LangChain provides a large collection of built-in tools for many common tasks. Here are some examples:

**Preparation:**
* Ensure you have `langchain-openai` and the necessary dependencies for each Tool installed.
* Set the `OPENAI_API_KEY` environment variable.
* For `SerpAPIWrapper`, you'll need a `SERPAPI_API_KEY`.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# Set environment variables for OpenAI and SerpAPI keys
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY" # Required for SerpAPIWrapper

### 3.1. `SerpAPIWrapper` (Web Search)

* **Concept:** Allows the Agent to perform web searches using SerpAPI (a paid API that provides search results from Google and other search engines).
* **When to Use:** When the Agent needs up-to-date information, factual data, or answers to questions not present in its training data.
* **Requirement:** `pip install google-search-results` and `SERPAPI_API_KEY`.

In [ ]:
# Install if not already installed:
# pip install google-search-results langchain

from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool

# Initialize SerpAPIWrapper
serpapi_wrapper = SerpAPIWrapper()

# Create Tool from SerpAPIWrapper
search_tool = Tool(
    name="Google Search",
    func=serpapi_wrapper.run,
    description="Useful when you need to search for information on Google about current events or factual data."
)

# Example of using the Tool directly (not through an Agent)
# print(search_tool.run("Weather in Da Nang today"))

### 3.2. `Calculator`

* **Concept:** A simple Tool to perform basic mathematical calculations.
* **When to Use:** When the Agent needs to perform arithmetic calculations that the LLM might struggle with or be unreliable at.
* **Requirement:** `pip install numexpr` (auxiliary library for Calculator).

In [ ]:
# Install if not already installed:
# pip install numexpr langchain

from langchain_community.tools.calculator.tool import Calculator

# Initialize Calculator Tool
calculator_tool = Calculator()

# Example of using the Tool directly
# print(calculator_tool.run("123 * 456"))

### 3.3. `WikipediaAPIWrapper` (Wikipedia Search)

* **Concept:** Allows the Agent to search and retrieve content from Wikipedia.
* **When to Use:** When the Agent needs encyclopedic information, definitions, or overviews of topics.
* **Requirement:** `pip install wikipedia`.

In [ ]:
# Install if not already installed:
# pip install wikipedia langchain

from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool

# Initialize WikipediaAPIWrapper
wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)

# Create Tool from WikipediaAPIWrapper
wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia_wrapper.run,
    description="Useful when you need to search for information on Wikipedia about concepts, people, or events."
)

# Example of using the Tool directly
# print(wikipedia_tool.run("LangChain"))

### 3.4. `PythonREPLTool` (Execute Python Code)

* **Concept:** Allows the Agent to execute Python code in a REPL (Read-Eval-Print Loop) environment.
* **When to Use:** When the Agent needs to perform programming tasks, data analysis, complex string manipulation, or any task that can be solved with Python code.
* **Warning:** This is a very powerful tool and poses security risks if not used carefully. It's not recommended to allow an Agent to run arbitrary user-provided code in a production environment without strict security measures.
* **Requirement:** `pip install langchain`.

In [ ]:
# Install if not already installed:
# pip install langchain

from langchain_community.tools.python.tool import PythonREPLTool

# Initialize PythonREPLTool
python_repl_tool = PythonREPLTool()

# Example of using the Tool directly
# print(python_repl_tool.run("print(2 + 2)"))
# print(python_repl_tool.run("import math; print(math.sqrt(16))"))


---

## 4. How to Connect Tools with an Agent

Once you have defined or initialized your Tools, the next step is to provide them to the Agent. When you initialize an Agent using `create_react_agent` (or other Agent creation functions), you will pass a list of Tool objects to it.

The Agent Executor will use the LLM to decide which Tool from this list to invoke based on the user's query and each Tool's description.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai google-search-results numexpr wikipedia

import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.utilities import SerpAPIWrapper, WikipediaAPIWrapper
from langchain.tools import Tool
from langchain_community.tools.calculator.tool import Calculator
from langchain_community.tools.python.tool import PythonREPLTool

# Set environment variables for OpenAI and SerpAPI keys
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # Lower temperature for more consistent Agent decisions

# Initialize Tools
search_tool = Tool(
    name="Google Search",
    func=SerpAPIWrapper().run,
    description="Useful when you need to search for information on Google about current events or factual data."
)

calculator_tool = Calculator()

wikipedia_tool = Tool(
    name="Wikipedia",
    func=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200).run,
    description="Useful when you need to search for information on Wikipedia about concepts, people, or events."
)

python_repl_tool = PythonREPLTool()

# Gather all Tools that the Agent can use
tools = [search_tool, calculator_tool, wikipedia_tool, python_repl_tool]

# Define Prompt for the Agent
# This prompt will instruct the LLM on how to think and use tools
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You have access to the following tools: {tools}. Use them to answer questions."),
    MessagesPlaceholder(variable_name="chat_history"), # To maintain chat history
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"), # For the Agent to record its thoughts and actions
])

# Create Agent
# 'react-json-chat-model' is an Agent type that allows the LLM to think and act (ReAct)
agent = create_react_agent(llm, tools, prompt)

# Create Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # verbose=True to see the Agent's thought process

# --- Execute the Agent ---
print("--- Starting Agent Executor ---")
chat_history = []

# Question 1: Requires web search
query_1 = "What's the weather like today in Da Nang?"
print(f"\nUser: {query_1}")
response_1 = agent_executor.invoke({"input": query_1, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_1), AIMessage(content=response_1["output"])])
print(f"Agent: {response_1['output']}")

# Question 2: Requires calculation
query_2 = "Calculate 15% of 3450."
print(f"\nUser: {query_2}")
response_2 = agent_executor.invoke({"input": query_2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_2), AIMessage(content=response_2["output"])])
print(f"Agent: {response_2['output']}")

# Question 3: Requires Wikipedia search
query_3 = "What is LangChain?"
print(f"\nUser: {query_3}")
response_3 = agent_executor.invoke({"input": query_3, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_3), AIMessage(content=response_3["output"])])
print(f"Agent: {response_3['output']}")

# Question 4: Requires Python execution
query_4 = "Write a Python code snippet to calculate the factorial of 5."
print(f"\nUser: {query_4}")
response_4 = agent_executor.invoke({"input": query_4, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_4), AIMessage(content=response_4["output"])])
print(f"Agent: {response_4['output']}")

print("--- Agent Executor Ended ---")

**Explanation:**
* `tools = [...]`: You create a list of Tool objects and pass it to the Agent.
* `create_react_agent(llm, tools, prompt)`: This function creates an Agent that uses the ReAct (Reasoning and Acting) prompting strategy with the given Tools and Prompt.
* `AgentExecutor(agent=agent, tools=tools, verbose=True)`: The Agent Executor is the orchestrating component. `verbose=True` is very useful for seeing the Agent's thought process and actions.
* When you call `agent_executor.invoke(...)`, the Agent will start its reasoning-action loop, select the appropriate Tool, execute it, and use the result to generate the final answer.


---

## Lesson Summary

This lesson delved into the concept of **Tools** in LangChain, which are functions or APIs that **Agents** use to extend the LLM's capabilities and interact with the external world. You learned how to **create custom tools** using the `tool` decorator and by inheriting from `BaseTool`. We also explored and practiced using several **common built-in tools** from LangChain such as `SerpAPIWrapper` (web search), `Calculator`, `WikipediaAPIWrapper`, and `PythonREPLTool`. Finally, you saw **how to connect these Tools to an Agent** and how the Agent Executor orchestrates the reasoning-action process to intelligently use tools to solve user problems. Mastering the use of Tools is key to building powerful and flexible Agents.